In [27]:
from bs4 import BeautifulSoup as bs
import requests
import re
import csv
import pandas as pd
from datetime import date
import datetime

In [9]:
'''
파이썬개발자
"https://www.wanted.co.kr/wdlist/518/899?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"
소프트웨어 엔지니어
"https://www.wanted.co.kr/wdlist/518/10110?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"
데이터 엔지니어
"https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"
데이터 사이언티스트
"https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"
빅데이터 엔지니어
"https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"
DBA
"https://www.wanted.co.kr/wdlist/518/10231?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"
'''

'\n파이썬개발자\n"https://www.wanted.co.kr/wdlist/518/899?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"\n소프트웨어 엔지니어\n"https://www.wanted.co.kr/wdlist/518/10110?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"\n데이터 엔지니어\n"https://www.wanted.co.kr/wdlist/518/655?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"\n데이터 사이언티스트\n"https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"\n빅데이터 엔지니어\n"https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"\nDBA\n"https://www.wanted.co.kr/wdlist/518/10231?country=kr&job_sort=company.response_rate_order&years=0&years=2&locations=all"\n'

In [10]:
"https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=company.response_rate_order&years=0&years=2&selected=10231&selected=655&locations=all"

'https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=company.response_rate_order&years=0&years=2&selected=10231&selected=655&locations=all'

In [11]:
job_keywords = {"sw", "ict", "it", "db", "보안", "정보", "인프라", "infra",
            "통신", "네트워크", "network", "data", "데이터", "db", "backend", "be",
            "백엔드", "server", "서버", "fe", "frontend", "프론트엔드", "프론트 엔드",
            "전산", "sw", "software", "소프트웨어", "개발", "시스템", "system", "dba",
            "cloud", "클라우드", "aws", "gcp", "컴퓨터", "엔지니어",
            "engineer", "s/w", "웹", "앱", "web", "app", "dx",
            "안드로이드", "임베디드", "php", "블록체인", "blockchain", "bi", "net"}
corp_keywords = {"스마일게이트", "은행", "cns"}


In [43]:
def get_stepup():
    url="https://cafe.naver.com/ArticleList.nhn?search.clubid=15754634&search.menuid=374&search.boardtype=L"
    response= requests.get(url)
    soup=bs(response.text,"html.parser")
    elements= soup.find_all("div","article-board m-tcol-c")[1]
    e=elements.table.tbody.find_all("tr")
    e=e[:5:2]
    url_list={}
    for row in e:
        temp={}
        title=re.sub(r'[^\w]','',row.td.find("div","inner_list").a.text)
        if "신입" in title:
            url_list["신입"]="https://cafe.naver.com/specup"+row.td.find("div","inner_list").a["href"]
        elif "인턴" in title:
            url_list["인턴"]="https://cafe.naver.com/specup"+row.td.find("div","inner_list").a["href"]
    
    response= requests.get(url_list["인턴"])
    soup=bs(response.text,"html.parser")
    divs= soup.find_all("div","tbody m-tcol-c")[0].div.find_all("div")
    img_link="https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fi.ibb.co%2FVH1ZBkT%2F5.png%22&type=cafe_wa740"
    for div in divs:
        if div.table:
            if div.table.tbody.tr.img["src"]==img_link:
                tbody = div.table.tbody

    intern_job_list=[]
        
    trs= list(map(lambda x:x.find_all("td"),tbody.find_all("tr")[2::2]))
    for tds in trs:
        url=tds[0].a["href"]
        temp={}
        temp["corporation_name"]=tds[0].a.text
        temp["job_name"]=tds[1].a.text
        if tds[2].span:
            temp["due_date"]=tds[2].span.text
        else:
            temp["due_date"]=tds[2].text
        temp["url"]=url
        intern_job_list.append(temp)
        
    response= requests.get(url_list["신입"])
    soup=bs(response.text,"html.parser")
    divs= soup.find_all("div","tbody m-tcol-c")[0].div.find_all("div")[2:]
    # divs
    tbody = divs[0].table.tbody
    big_job_list=[]
    temp= list(map(lambda x:x.find_all("td"),tbody.find_all("tr")[2::2]))
    trs=[]
    for tr in temp:
        if tr:
            trs.append(tr)
            
    for tds in trs:
        url=tds[0].a["href"]
        temp={}
        temp["corporation_name"]=tds[0].a.text
        temp["job_name"]=tds[1].a.text
        if tds[2].span:
            temp["due_date"]=tds[2].span.text
        else:
            temp["due_date"]=tds[2].text
        temp["url"]=url
        big_job_list.append(temp)

    tbody = divs[1].table.tbody
    middle_job_list=[]
    temp= list(map(lambda x:x.find_all("td"),tbody.find_all("tr")[2::2]))
    trs=[]
    for tr in temp:
        if tr:
            trs.append(tr)
    for tds in trs:
        url=tds[0].a["href"]
        temp={}
        temp["corporation_name"]=tds[0].a.text
        temp["job_name"]=tds[1].a.text
        if tds[2].span:
            temp["due_date"]=tds[2].span.text
        else:
            temp["due_date"]=tds[2].text
        temp["url"]=url
        middle_job_list.append(temp)

    tbody = divs[2].table.tbody
    popular_job_list=[]
    temp= list(map(lambda x:x.find_all("td"),tbody.find_all("tr")[2::2]))
    trs=[]
    for tr in temp:
        if tr:
            trs.append(tr)
    for tds in trs:
        url=tds[0].a["href"]
        temp={}
        temp["corporation_name"]=tds[0].a.text
        temp["job_name"]=tds[1].a.text
        if tds[2].span:
            temp["due_date"]=tds[2].span.text
        else:
            temp["due_date"]=tds[2].text
        temp["url"]=url
        popular_job_list.append(temp)
    
    return intern_job_list, big_job_list, middle_job_list, popular_job_list
            
intern_job_list, big_job_list, middle_job_list, popular_job_list = get_stepup()

In [44]:
filtered_big_job_list = []
for posting in big_job_list:
    sentence = posting["job_name"].lower()
    for job_keyword in job_keywords:
        if job_keyword in sentence:
            filtered_big_job_list.append(posting)
            break
        
filtered_middle_job_list = []
for posting in middle_job_list:
    sentence = posting["job_name"].lower()
    for job_keyword in job_keywords:
        if job_keyword in sentence:
            filtered_middle_job_list.append(posting)
            break
        
     
# intern_job_list, filtered_big_job_list, filtered_middle_job_list, popular_job_list

In [45]:
my_date = date(2023, 7, 1)
now = datetime.datetime.now()
today_year = now.year
today_month = now.month
today_day = now.day

for i in range(len(intern_job_list)):
    posting = intern_job_list[i]
    due_date = posting["due_date"]
    if "~" in due_date:
        month, day = map(int,due_date.strip()[1:].split("/"))
        if month < today_month:
            year = today_year + 1
        else:
            year = today_year
        intern_job_list[i]["due_date"] = date(year, month, day)
    else:
        intern_job_list[i]["due_date"] = None

for i in range(len(intern_job_list)):
    posting = intern_job_list[i]
    url = posting["url"]
    intern_job_list[i]["STEPUP_POSTING_ID"] = int(url.split("/")[-1])
    del intern_job_list[i]["url"]
    

for i in range(len(intern_job_list)):
    intern_job_list[i]["POSITION_ID"] = 0
intern_job_list

        

[{'corporation_name': '토스증권',
  'job_name': ' 2023 Data Analyst Internship 채용 ',
  'due_date': datetime.date(2023, 7, 12),
  'STEPUP_POSTING_ID': 7301387,
  'POSITION_ID': 0},
 {'corporation_name': '아드반테스트코리아',
  'job_name': ' SoC application engineer 인턴 채용 ',
  'due_date': datetime.date(2023, 7, 3),
  'STEPUP_POSTING_ID': 7301128,
  'POSITION_ID': 0},
 {'corporation_name': '창신INC',
  'job_name': ' 2023년 7월 Digital PCC 채용형 인턴 채용 ',
  'due_date': datetime.date(2023, 7, 11),
  'STEPUP_POSTING_ID': 7300781,
  'POSITION_ID': 0},
 {'corporation_name': '데브시스터즈',
  'job_name': ' 쿠키런 오븐브레이크 맵 제작 및 밸런스 기획자(인턴) 채용 ',
  'due_date': None,
  'STEPUP_POSTING_ID': 7300550,
  'POSITION_ID': 0},
 {'corporation_name': '캐시워크',
  'job_name': ' 백엔드개발 채용전환형 인턴 채용 ',
  'due_date': datetime.date(2023, 7, 11),
  'STEPUP_POSTING_ID': 7300540,
  'POSITION_ID': 0},
 {'corporation_name': '캐시워크',
  'job_name': ' 프론트엔드개발 채용전환형 인턴 채용 ',
  'due_date': datetime.date(2023, 7, 11),
  'STEPUP_POSTING_ID': 7300539,
  'POSI

In [46]:
my_date = date(2023, 7, 1)
now = datetime.datetime.now()
today_year = now.year
today_month = now.month
today_day = now.day

for i in range(len(filtered_big_job_list)):
    posting = filtered_big_job_list[i]
    due_date = posting["due_date"]
    if "~" in due_date:
        month, day = map(int,due_date.strip()[1:].split("/"))
        if month < today_month:
            year = today_year + 1
        else:
            year = today_year
        filtered_big_job_list[i]["due_date"] = date(year, month, day)
    else:
        filtered_big_job_list[i]["due_date"] = None

for i in range(len(filtered_big_job_list)):
    posting = filtered_big_job_list[i]
    url = posting["url"]
    filtered_big_job_list[i]["STEPUP_POSTING_ID"] = int(url.split("/")[-1])
    del filtered_big_job_list[i]["url"]
    

for i in range(len(filtered_big_job_list)):
    filtered_big_job_list[i]["POSITION_ID"] = 1
filtered_big_job_list

        

[{'corporation_name': 'CJ올리브네트웍스',
  'job_name': ' 방송 송출 운영 및 시스템 관리 신입 채용 ',
  'due_date': datetime.date(2023, 7, 16),
  'STEPUP_POSTING_ID': 7301438,
  'POSITION_ID': 1},
 {'corporation_name': '현대오토에버',
  'job_name': ' 차량SW, ICT 부문 경력 채용 ',
  'due_date': None,
  'STEPUP_POSTING_ID': 7301084,
  'POSITION_ID': 1},
 {'corporation_name': 'AK아이에스',
  'job_name': ' 애경그룹 2023년 하반기 DB운영, 항공시스템 운영 등 각 부문별 신입/경력 채용 ',
  'due_date': datetime.date(2023, 7, 13),
  'STEPUP_POSTING_ID': 7300734,
  'POSITION_ID': 1},
 {'corporation_name': 'SK쉴더스',
  'job_name': ' SK그룹사 파주지사 시설보안직 신규채용 ',
  'due_date': None,
  'STEPUP_POSTING_ID': 7299976,
  'POSITION_ID': 1},
 {'corporation_name': '호반호텔앤리조트',
  'job_name': ' IT, 마케팅 등 부문별 신입/경력사원 채용 ',
  'due_date': datetime.date(2023, 7, 9),
  'STEPUP_POSTING_ID': 7299656,
  'POSITION_ID': 1},
 {'corporation_name': '포스코DX',
  'job_name': ' 정보보안 분야 신입/경력 채용 ',
  'due_date': datetime.date(2023, 7, 9),
  'STEPUP_POSTING_ID': 7299235,
  'POSITION_ID': 1},
 {'corporatio

In [47]:
my_date = date(2023, 7, 1)
now = datetime.datetime.now()
today_year = now.year
today_month = now.month
today_day = now.day

for i in range(len(filtered_middle_job_list)):
    posting = filtered_middle_job_list[i]
    due_date = posting["due_date"]
    if "~" in due_date:
        month, day = map(int,due_date.strip()[1:].split("/"))
        if month < today_month:
            year = today_year + 1
        else:
            year = today_year
        filtered_middle_job_list[i]["due_date"] = date(year, month, day)
    else:
        filtered_middle_job_list[i]["due_date"] = None

for i in range(len(filtered_middle_job_list)):
    posting = filtered_middle_job_list[i]
    url = posting["url"]
    filtered_middle_job_list[i]["STEPUP_POSTING_ID"] = int(url.split("/")[-1])
    del filtered_middle_job_list[i]["url"]
    

for i in range(len(filtered_middle_job_list)):
    filtered_middle_job_list[i]["POSITION_ID"] = 1
filtered_middle_job_list

        

[{'corporation_name': '인성',
  'job_name': ' 전산팀, AP영업팀 등 각 부문별 채용 ',
  'due_date': datetime.date(2023, 7, 30),
  'STEPUP_POSTING_ID': 7301411,
  'POSITION_ID': 1},
 {'corporation_name': '한국자산평가',
  'job_name': ' 파생평가,솔루션,IT인프라사업 부문 신입(인턴) 및 경력직원 채용 ',
  'due_date': datetime.date(2023, 7, 16),
  'STEPUP_POSTING_ID': 7301127,
  'POSITION_ID': 1},
 {'corporation_name': '피아이첨단소재',
  'job_name': ' 안전환경, 생산엔지니어 부문 신입사원 채용 ',
  'due_date': datetime.date(2023, 7, 9),
  'STEPUP_POSTING_ID': 7301064,
  'POSITION_ID': 1},
 {'corporation_name': '강남제비스코',
  'job_name': ' 연구개발 기술 신입사원 채용 ',
  'due_date': datetime.date(2023, 7, 9),
  'STEPUP_POSTING_ID': 7300787,
  'POSITION_ID': 1},
 {'corporation_name': '다원시스',
  'job_name': ' 2023년 상반기 전기/전자/제어, 소프트웨어 등 각 분야별 신입/경력 공개채용 ',
  'due_date': datetime.date(2023, 7, 27),
  'STEPUP_POSTING_ID': 7300784,
  'POSITION_ID': 1},
 {'corporation_name': '태양금속공업',
  'job_name': ' 개발팀 신입/경력사원 채용 ',
  'due_date': datetime.date(2023, 7, 23),
  'STEPUP_POSTING_ID': 73

In [49]:
my_date = date(2023, 7, 1)
now = datetime.datetime.now()
today_year = now.year
today_month = now.month
today_day = now.day

for i in range(len(popular_job_list)):
    posting = popular_job_list[i]
    due_date = posting["due_date"]
    if "~" in due_date:
        month, day = map(int,due_date.strip()[1:].split("/"))
        if month < today_month:
            year = today_year + 1
        else:
            year = today_year
        popular_job_list[i]["due_date"] = date(year, month, day)
    else:
        popular_job_list[i]["due_date"] = None

for i in range(len(popular_job_list)):
    posting = popular_job_list[i]
    url = posting["url"]
    popular_job_list[i]["STEPUP_POSTING_ID"] = int(url.split("/")[-1])
    del popular_job_list[i]["url"]
    

for i in range(len(popular_job_list)):
    popular_job_list[i]["POSITION_ID"] = 2
popular_job_list

        

[{'corporation_name': 'HDC리조트',
  'job_name': ' 내부통제팀 IT 신입 채용 ',
  'due_date': datetime.date(2023, 7, 29),
  'STEPUP_POSTING_ID': 7301454,
  'POSITION_ID': 2},
 {'corporation_name': '소노인터내셔널',
  'job_name': ' 전기/환경 채용 ',
  'due_date': datetime.date(2023, 7, 6),
  'STEPUP_POSTING_ID': 7301448,
  'POSITION_ID': 2},
 {'corporation_name': 'SG생활안전',
  'job_name': ' 전문건설업 관리 및 사무지원 채용 ',
  'due_date': datetime.date(2023, 7, 29),
  'STEPUP_POSTING_ID': 7301446,
  'POSITION_ID': 2},
 {'corporation_name': '이글루코퍼레이션',
  'job_name': ' 악성코드분석 경력 채용 ',
  'due_date': datetime.date(2023, 7, 31),
  'STEPUP_POSTING_ID': 7301442,
  'POSITION_ID': 2},
 {'corporation_name': '하이비젼시스템',
  'job_name': ' 설비 SW개발자 신입 채용 ',
  'due_date': datetime.date(2023, 7, 29),
  'STEPUP_POSTING_ID': 7301440,
  'POSITION_ID': 2},
 {'corporation_name': 'CJ대한통운',
  'job_name': ' PD 정비계약직 채용 ',
  'due_date': datetime.date(2023, 7, 9),
  'STEPUP_POSTING_ID': 7301437,
  'POSITION_ID': 2},
 {'corporation_name': '태웅',
  'job_name

In [54]:
job_list = sorted((intern_job_list + filtered_big_job_list + filtered_middle_job_list + popular_job_list), key = lambda x:x["STEPUP_POSTING_ID"])
temp = []
before = -999
for i in range(len(job_list)):
    if job_list[i]["STEPUP_POSTING_ID"] != before:
        temp.append(job_list[i])
    else:
        continue
print(len(job_list))
print(len(temp))
job_list = temp   
    

84
84


In [58]:
df = pd.DataFrame(job_list)
df = df[["STEPUP_POSTING_ID", "corporation_name", "job_name", "due_date", "POSITION_ID"]]
df.rename(columns = {'corporation_name' : 'CORPORATION_NAME'}, inplace = True)
df.rename(columns = {'job_name' : 'JOB_NAME'}, inplace = True)
df.rename(columns = {'due_date' : 'DUE_DATE'}, inplace = True)
df.rename(columns = {'due_date' : 'DUE_DATE'}, inplace = True)
df


,STEPUP_POSTING_ID,CORPORATION_NAME,JOB_NAME,DUE_DATE,POSITION_ID
0,7297837,SPC삼립,"마케팅, 연구개발, 상품기획 등 부문별 채용",2024-06-28,1
1,7298464,파수,"데이터 보안 및 개인정보보호, 문서관리 및 협업 등 분야별 신입 공개 채용",2023-07-09,1
2,7298871,화신그룹,"2023년 상반기 엔지니어, 품질보증, 안전관리 등 부문별 신입사원 채용",2023-07-16,1
3,7298882,이화다이아몬드공업,"2023년 연구개발, 영업 등 부문별 신입 및 경력 수시 채용",2023-07-05,1
4,7298885,해태제과,"디자인, 개발, 품질 등 부문별 채용연계형 인턴 및 경력직 채용",2023-07-04,1
...,...,...,...,...,...
79,7301440,하이비젼시스템,설비 SW개발자 신입 채용,2023-07-29,2
80,7301442,이글루코퍼레이션,악성코드분석 경력 채용,2023-07-31,2
81,7301446,SG생활안전,전문건설업 관리 및 사무지원 채용,2023-07-29,2
82,7301448,소노인터내셔널,전기/환경 채용,2023-07-06,2
